In [ ]:
import os, time, random
import numpy as np
import pandas as pd
import cv2, torch
from tqdm.auto import tqdm
import shutil as sh

from IPython.display import Image, clear_output
import matplotlib.pyplot as plt


In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -U pycocotools
!pip install -qr yolov5/requirements.txt  # install dependencies
!cp yolov5/requirements.txt ./

In [ ]:
img_h, img_w, num_channels = (380, 676, 3)
df = pd.read_csv("/kaggle/input/car-object-detection/data/train_solution_bounding_boxes (1).csv")

# change column name
df.rename(columns={'image':'image_id'}, inplace=True)
df['image_id'] = df['image_id'].apply(lambda x: x.split('.')[0])

# preprocess the data for yolo object detection
df['x_center'] = (df['xmin'] + df['xmax'])/2
df['y_center'] = (df['ymin'] + df['ymax'])/2
df['w'] = df['xmax'] - df['xmin']
df['h'] = df['ymax'] - df['ymin']
df['classes'] = 0
df['status'] = 1

# scale the data
df['x_center'] = df['x_center'] / img_w
df['w'] = df['w'] / img_w
df['y_center'] = df['y_center'] / img_h
df['h'] = df['h'] / img_h

df.head()

#### Explain 

In [ ]:
if not os.path.exists(os.path.join('mydata', 'images')):
    os.makedirs(os.path.join('mydata', 'images'))

if not os.path.exists(os.path.join('mydata', 'labels')):
    os.makedirs(os.path.join('mydata', 'labels'))

for idx, data in df.groupby('image_id'):
    row = data[['classes', 'x_center', 'y_center', 'w', 'h']].astype(float).values
    row = row.astype(str) # ['0.0', '0.45043415340236687', '0.539816602368421','0.06874095502958583', '0.09523809526315791']
    
    with open('mydata/labels/{}.txt'.format(idx), 'w+') as f:
        for i in range(len(row)):
            text = ' '.join(row[i]) #0.0 0.45043415340236687 0.539816602368421 0.06874095502958583 0.09523809526315791
            f.write(text)
            f.write('\n')

    sh.copy(
      '/kaggle/input/car-object-detection/data/training_images/{}.jpg'.format(idx),
      'mydata/images/{}.jpg'.format(idx)
    )

In [ ]:
with open('yolov5/dataset.yaml', 'w+') as f:
    f.write('path: ../mydata\n')
    f.write('train: images\n')
    f.write('val: images\n\n')
    f.write('names:\n')
    f.write('  0: car')

In [ ]:
cd yolov5

In [ ]:
!python train.py --img 676 --batch 16 --epochs 20 --data dataset.yaml --weights yolov5s.pt --workers 2

In [ ]:
cd ..

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/kaggle/working/yolov5/runs/train/exp/weights/best.pt', force_reload=True)

In [ ]:
img = os.path.join('mydata', 'images', 'vid_4_26460.jpg')
results = model(img)
results.print()

In [ ]:
%matplotlib inline
plt.imshow(np.squeeze(results.render()))
plt.show()


In [ ]:
img = "/kaggle/input/car-object-detection/data/testing_images/vid_5_26320.jpg"

In [ ]:
plt.imshow(np.squeeze(model(img).render()))
plt.show()

In [ ]:
def detect_vehicle(image_path):
    results = model(image_path)
    return np.squeeze(results.render())

In [ ]:
import os
import matplotlib.pyplot as plt

folder_dir = "/kaggle/input/car-object-detection/data/testing_images"
image_files = [f for f in os.listdir(folder_dir) if f.lower().endswith((".png", ".jpg", ".jpeg"))]

# Create subplots with a 2xN grid layout
# num_images = len(image_files)
# columns, rows = 2, (num_images + 1) // 2
# fig, axs = plt.subplots(rows, columns, figsize=(100, 100))

for i, image_file in enumerate(image_files):
    img_path = os.path.join(folder_dir, image_file)
    print(img_path)
    img = detect_vehicle(img_path)
    plt.imshow(img)
    plt.show()
#     ax = axs[i // columns, i % columns]
#     ax.imshow(img)
# #     ax.set_title(image_file)  # Set image filename as title
#     ax.axis("off")  # Hide axes

plt.tight_layout()
plt.show()
